In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yfinance as yf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

SYMBOL = "TSLA"
START = "2020-01-01"
END = "2021-01-01"
window = 100

In [ ]:
data = yf.Ticker(SYMBOL).history(start=START, end=END)[["Close"]]

In [ ]:
returns = np.log(data["Close"]).diff().dropna()

forecast_len = len(returns) - window
forecasts = np.zeros(forecast_len)
directions = np.zeros(forecast_len)
vol_forecasts = np.zeros(forecast_len)

for i in range(window, len(returns) - 1):
    print(f"Step {i}/{len(returns)-1}", end="\r")
    train = returns.iloc[i-window:i]

    bic = []
    orders_list = []
    model_list = []

    for p in range(1, 4):
        for q in range(1, 4):
            try:
                model = ARIMA(train, order=(p, 0, q))
                model_fit = model.fit()
                bic.append(model_fit.bic)
                orders_list.append((p, q))
                model_list.append(model_fit)
            except:
                bic.append(np.inf)
                orders_list.append((p, q))
                model_list.append(None)

    idx = int(np.argmin(bic))
    best_model = model_list[idx]
    best_p, best_q = orders_list[idx]

    if best_model is None:
        continue

    arma_forecast = best_model.get_forecast(steps=1).predicted_mean.iloc[0]

    residuals = best_model.resid
    garch_model = arch_model(residuals, vol="GARCH", p=1, q=1, mean="Zero", dist="t")
    garch_fit = garch_model.fit(disp="off")

    garch_forecast = garch_fit.forecast(horizon=1)
    vol_forecast = np.sqrt(garch_forecast.variance.iloc[-1, 0])

    tmp = i - window
    forecasts[tmp] = arma_forecast
    directions[tmp] = 1 if arma_forecast > 0 else -1
    vol_forecasts[tmp] = vol_forecast




In [ ]:
strategy = pd.DataFrame({
    "f_Mean": forecasts,
    "f_Direction": directions,
    "f_Volatility": vol_forecasts
}, index=returns.index[window:len(returns)])

strategy["Returns"] = strategy["f_Direction"] * returns.iloc[window:]

curve = strategy["Returns"].cumsum()
long_curve = returns.iloc[window+1:].cumsum()

total_return = pd.DataFrame({
    "Strategy": curve,
    "Buy&Hold": long_curve
})
total_return.plot(title=f"{SYMBOL} Strategy vs Buy&Hold", figsize=(12,6))
plt.show()